In [2]:
import pandas as pd
import mysql.connector

from IPython.display import display, Markdown, HTML

db = mysql.connector.connect(
    host="localhost",
    user="root",
    password="secret",
    port="3306",
    database="serlo"
)

/home/kulla/.local/share/asdf/installs/python/3.9.1/lib/python3.9/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## Liste fehlerhafter Events

In [10]:
df = pd.read_sql("""
    select
        event_log.id as event_log_id,
        event_log.uuid_id as event_log_uuid,
        event_parameter.id as event_parameter_id,
        event_parameter_uuid.id as event_parameter_uuid_id,
        event_parameter_uuid.uuid_id as event_parameter_uuid_uuid
    from event_log
    join event_parameter on event_parameter.log_id = event_log.id
    join event_parameter_uuid on event_parameter_uuid.event_parameter_id = event_parameter.id
    join uuid on uuid.id = event_log.uuid_id
    where event_log.event_id = 9
        and uuid.discriminator != "comment";
""", db)
df

,event_log_id,event_log_uuid,event_parameter_id,event_parameter_uuid_id,event_parameter_uuid_uuid
0,405293,184439,415303,305286,212405
1,406265,170410,416305,305986,212994
2,406273,174234,416311,305992,212999
3,409132,185201,418807,307871,214551
4,410231,21019,419966,308669,215064
5,412089,179318,421896,310007,216474
6,412779,192412,422309,310383,216909
7,413198,192412,422568,310618,217154
8,413208,152489,422577,310625,217169
9,413571,202923,422857,310844,217337


### Generierung der Updates

In [27]:
for i, row in df.iterrows():
    # Swap uuid and uuid_parameter
    print("update event_parameter_uuid set uuid_id = "
          + str(row["event_log_uuid"])
          + " where id = "
          + str(row["event_parameter_uuid_id"])
          + ";")
    
    print("update event_log set uuid_id = "
         + str(row["event_parameter_uuid_uuid"])
         + " where id = "
         + str(row["event_log_id"])
         + ";")
    
    # Fix event_paramater_name to "on"
    print("update event_parameter set name_id = 4 where id = "
         + str(row["event_parameter_id"]) + ";")
    
    # Fix event_type to create_thread
    print("update event_log set event_id = 8 where id = "
         + str(row["event_log_id"]) + ";")

update event_parameter_uuid set uuid_id = 184439 where id = 305286;
update event_log set uuid_id = 212405 where id = 405293;
update event_parameter set name_id = 4 where id = 415303;
update event_log set event_id = 8 where id = 405293;
update event_parameter_uuid set uuid_id = 170410 where id = 305986;
update event_log set uuid_id = 212994 where id = 406265;
update event_parameter set name_id = 4 where id = 416305;
update event_log set event_id = 8 where id = 406265;
update event_parameter_uuid set uuid_id = 174234 where id = 305992;
update event_log set uuid_id = 212999 where id = 406273;
update event_parameter set name_id = 4 where id = 416311;
update event_log set event_id = 8 where id = 406273;
update event_parameter_uuid set uuid_id = 185201 where id = 307871;
update event_log set uuid_id = 214551 where id = 409132;
update event_parameter set name_id = 4 where id = 418807;
update event_log set event_id = 8 where id = 409132;
update event_parameter_uuid set uuid_id = 21019 where id

In [28]:
for i, row in df.iterrows():
    print("redis_del('de.serlo.org/api/event/" + str(row["event_log_id"]) + "');")

redis_del('de.serlo.org/api/event/405293');
redis_del('de.serlo.org/api/event/406265');
redis_del('de.serlo.org/api/event/406273');
redis_del('de.serlo.org/api/event/409132');
redis_del('de.serlo.org/api/event/410231');
redis_del('de.serlo.org/api/event/412089');
redis_del('de.serlo.org/api/event/412779');
redis_del('de.serlo.org/api/event/413198');
redis_del('de.serlo.org/api/event/413208');
redis_del('de.serlo.org/api/event/413571');
redis_del('de.serlo.org/api/event/413615');
redis_del('de.serlo.org/api/event/415029');
redis_del('de.serlo.org/api/event/415048');
redis_del('de.serlo.org/api/event/417535');
